In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from numpy.random import seed

import pandas as pd
import numpy as np
import string, os 

In [ ]:
!pip install spacy_hunspell

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy_hunspell import spaCyHunSpell

In [ ]:
nlp = spacy.load('en_core_web_sm')
hunspell = spaCyHunSpell(nlp, ('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff'))

In [ ]:
df = pd.read_csv('../input/qa-csv/qa.csv', delimiter='\t', encoding='utf-8', index_col=0)

In [ ]:
df.columns

In [ ]:
df['Query'] = df['Query'].fillna('')

In [ ]:
df

In [ ]:
queries = list(df['Query'])

In [ ]:
queries

In [ ]:
queries_updated = []
for query in queries:
    query = query.strip()
    split_len = len(query.split(' '))
    if split_len >= 2:
        queries_updated.append(query)

In [ ]:
len(queries_updated)

In [ ]:
# df = df.drop(['Query'], axis=1)
del df

In [ ]:
df = pd.DataFrame(queries_updated, columns=['Query'])

In [ ]:
# df.reset_index()

In [ ]:
df.columns

In [ ]:
df.columns

In [ ]:
len(df)

In [ ]:
tokenizer = Tokenizer()
# def get_sequence_of_tokens(corpus):
#     pass
tokenizer.fit_on_texts(queries_updated)

In [ ]:
total_words = len(tokenizer.word_index) + 1
total_words

In [ ]:
input_sequences = []
for line in queries_updated:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
input_sequences

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
for i, j in zip(predictors, label):
    print(i, j)
label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
queries_updated

In [ ]:
queries_updated_tags = []

In [ ]:
for query in queries_updated:
    nlp_text = nlp(query)
    token_list = []
    for token in nlp_text:
        token_list.append(token.pos_)
    nlp_pos = " ".join(token_list)
    queries_updated_tags.append(nlp_pos)

In [ ]:
queries_updated_tags

In [ ]:
tokenizer_pos = Tokenizer()
tokenizer_pos.fit_on_texts(queries_updated_tags)

In [ ]:
total_words_pos = len(tokenizer_pos.word_index) + 1
total_words_pos

In [ ]:
input_sequences_pos = []
for line in queries_updated_tags:
    token_list = tokenizer_pos.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences_pos.append(n_gram_sequence)

In [ ]:
input_sequences_pos

In [ ]:
input_sequences_pos

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences_pos])
input_sequences_pos = np.array(pad_sequences(input_sequences_pos, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences_pos[:,:-1],input_sequences_pos[:,-1]
for i, j in zip(predictors, label):
    print(i, j)
label = ku.to_categorical(label, num_classes=total_words_pos)

In [ ]:
len(label)

In [ ]:
print(label[1])

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
    
    return model

In [ ]:
model = create_model(max_sequence_len, total_words_pos)
model.summary()

In [ ]:
model.fit(predictors, label, epochs=5)

In [ ]:
def convert_into_pos_tag(query):
    nlp_text = nlp(query)
    token_list = []
    for token in nlp_text:
        token_list.append(token.pos_)
    nlp_pos = " ".join(token_list)
    return nlp_pos

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    seed_text = convert_into_pos_tag(seed_text)
    print(seed_text)
    for _ in range(next_words):
        token_list = tokenizer_pos.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_freq = model.predict(token_list)
        print(predicted_freq)
        predicted = model.predict_classes(token_list, verbose=0)
        print(predicted)
        output_word = ""
        for word,index in tokenizer_pos.word_index.items():
            if index == predicted:
                print('index:', index, 'type of index:', type(index), 'predicted:', predicted,  'type of predicted:', type(predicted))
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
seed_text = "Short"

In [ ]:
generate_text(seed_text, 1, model, max_sequence_len)

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences_pos])
input_sequences_pos = np.array(pad_sequences(input_sequences_pos, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences_pos[:,:-1],input_sequences_pos[:,1:]
for i, j in zip(predictors, label):
    print(i, j)
# print(label)
label = ku.to_categorical(label, num_classes=total_words_pos)

In [ ]:
label[0]

In [ ]:
label.shape

In [ ]:
len(label)

In [ ]:
def create_model_time_sequenced(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(TimeDistributed(Dense(total_words, activation='softmax')))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
    
    return model

In [ ]:
model_time_sequenced = create_model_time_sequenced(max_sequence_len, total_words_pos)
model_time_sequenced.summary()

In [ ]:
model_time_sequenced.fit(predictors, label, epochs=5)

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    seed_text = convert_into_pos_tag(seed_text)
    print(seed_text)
    for _ in range(next_words):
        token_list = tokenizer_pos.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        print('tok list:', token_list)
        predicted_freq = model.predict(token_list)
        print(predicted_freq)
        prediction_probs = []
        for lev_1 in predicted_freq:
            for lev_2 in lev_1:
                print(np.max(lev_2))
        predicted = model.predict_classes(token_list, verbose=0)
        print(predicted)
        output_word = ""
        for pred in predicted:
            for ind_pred in pred:
                for word,index in tokenizer_pos.word_index.items():
                    if index == ind_pred:
                        output_word = word
                        break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
seed_text = "black long sleeves jerkin coat"

In [ ]:
generate_text(seed_text, 1, model_time_sequenced, max_sequence_len)

In [ ]:
a = np.array([1])

In [ ]:
type(a)

In [ ]:
a

In [ ]:
a == 1